In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [84]:
#Getting URL
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
positions = ["running-back", "wide-receiver"]
#Create an empty dataframe
data_frame = []

#I used 2 for loops to get the URL for each year and position that I want
for position in positions:
    for year in years:
        #gets the url for both the position and year for each team
        url = f"https://overthecap.com/position/{position}/{year}"
        #define the get request for the url
        req = requests.get(url)
        #Uses beautiful soup to parse the text in our url
        soup = BeautifulSoup(req.text, 'html.parser')
        #Finds the part of the HTML titled table. This is where all of the data I want to scrape is.
        table = soup.find('table')
        #Scrapes the data from the table into a dataframe
        salary_df = pd.read_html(str(table))[0]
        #Add both Position and year variables to the dataframe
        salary_df["Position"] = position
        salary_df["Year"] = year
        #Append the new data each time the for loops run
        data_frame.append(salary_df)

#Merge all of the data into one dataframe
salary_df = pd.concat(data_frame)

/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1648213898.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  salary_df = pd.read_html(str(table))[0]
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1648213898.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  salary_df = pd.read_html(str(table))[0]
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1648213898.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  salary_df = pd.read_html(str(table))[0]
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1648213898.py:19: FutureWarning: Passing literal html to 

In [ ]:
salary_df.head(10)
salary_df.tail(10)

#Clean Cash Spent to be in integer, first getting rid of $ and commas
salary_df["Cash Spent"] = (
    salary_df["Cash Spent"]
    .replace('[\$,]', '', regex=True)     
    .astype(int)
)

#same thing for cap number
salary_df["Cap Number"] = (
    salary_df["Cap Number"]
    .replace('[\$,]', '', regex=True)     # remove $ and commas
    .astype(int)
)

#len(salary_df)

<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1284380550.py:6: SyntaxWarning: invalid escape sequence '\$'
  .replace('[\$,]', '', regex=True)     # remove $ and commas
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_9722/1284380550.py:12: SyntaxWarning: invalid escape sequence '\$'
  .replace('[\$,]', '', regex=True)     # remove $ and commas


In [91]:
#Code to limit salary data to highest paid receiver (WR1) and highest paid back (RB1)

top_salaries = (
    salary_df
    .sort_values(["Position", "Year", "Team", "Cash Spent"], ascending=[True, True, True, False])
    .drop_duplicates(subset=["Position", "Year", "Team"])
)
#I create a new dataframe where only the WR1 and RB1 are included for each team. I used groupby to 
#apply the code to each year, postion, and team, used .loc and .idxmac to find the most cash spent 
#per each team and position per year and dropped all other values


top_salaries.tail(16)

,Player,Team,Cap Number,Cash Spent,Position,Year
32,Darius Slayton,Giants,6970593,13000005,wide-receiver,2025
23,Travis Hunter,Jaguars,8481657,31406628,wide-receiver,2025
31,Garrett Wilson,Jets,7032092,15000000,wide-receiver,2025
11,Amon-Ra St. Brown,Lions,13910000,17410000,wide-receiver,2025
61,Matthew Golden,Packers,3195516,10262064,wide-receiver,2025
42,Tetairoa McMillan,Panthers,5078253,17793012,wide-receiver,2025
21,Stefon Diggs,Patriots,8700000,18300000,wide-receiver,2025
105,Jack Bech,Raiders,1374331,2977324,wide-receiver,2025
14,Davante Adams,Rams,12000000,20000000,wide-receiver,2025
44,Rashod Bateman,Ravens,4798333,10000000,wide-receiver,2025


In [72]:
#Using downloaded csv files to create season dataframe and adding a year variable
season_df = []

#for loop with each year
for year in years:
    #reads csv files by year
    new_df = pd.read_csv(f"season{year}.csv")
    #adds an integer variable type
    new_df["Year"] = int(year)
    season_df.append(new_df)

#puts all data into one df
season_df = pd.concat(season_df)


In [51]:
season_df.tail(32)

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,Year,Pts,Pts.1
0,New England Patriots*,8,2,0.0,0.800,265.0,192.0,73,7.3,-4.3,3.0,2.7,0.3,2025,NaN,NaN
1,Buffalo Bills+,6,3,0.0,0.667,248.0,197.0,51,5.7,-3.0,2.7,3.7,-1.0,2025,NaN,NaN
2,Miami Dolphins,3,7,0.0,0.300,210.0,256.0,-46,-4.6,-0.7,-5.3,-2.9,-2.4,2025,NaN,NaN
3,New York Jets,2,7,0.0,0.222,195.0,241.0,-46,-5.1,-3.7,-8.8,-4.0,-4.8,2025,NaN,NaN
4,Pittsburgh Steelers*,5,4,0.0,0.556,212.0,220.0,-8,-0.9,-0.7,-1.6,-0.7,-0.9,2025,NaN,NaN
5,Baltimore Ravens,4,5,0.0,0.444,229.0,235.0,-6,-0.7,2.0,1.4,3.7,-2.3,2025,NaN,NaN
6,Cincinnati Bengals,3,6,0.0,0.333,216.0,300.0,-84,-9.3,-1.5,-10.8,0.5,-11.3,2025,NaN,NaN
7,Cleveland Browns,2,7,0.0,0.222,146.0,211.0,-65,-7.2,-1.4,-8.6,-8.9,0.3,2025,NaN,NaN
8,Indianapolis Colts*,8,2,0.0,0.800,321.0,206.0,115,11.5,-2.2,9.3,9.2,0.0,2025,NaN,NaN
9,Jacksonville Jaguars+,5,4,0.0,0.556,205.0,220.0,-15,-1.7,3.0,1.4,2.1,-0.8,2025,NaN,NaN


In [73]:
#cleaning season df

#This code assigns a column named Team where only the last element of the team name is taken
#to do this I used .str.split and str[-1] to get the last string element
season_df["Team"] = season_df["Tm"].str.split().str[-1]
#str.strip was used to get rid of playoff clinching indicators
season_df["Team"] = season_df["Team"].str.strip("*+")

#Created a new column titled win_pct
season_df["win_pct"] = season_df["W-L%"]

#Created a win - loss differential variable to use for exploratory data analysis
season_df["win_loss_diff"] = season_df["W"] - season_df["L"]


season_df["T"] = pd.to_numeric(season_df["T"], errors="coerce").fillna(0).astype(int)

season_df = season_df[["Team", "W", "L", "T", "win_pct", "win_loss_diff", "PF", "Year"]]


In [74]:
season_df.head(5)

,Team,W,L,T,win_pct,win_loss_diff,PF,Year
0,Patriots,12,4,0,0.750,8,420.0,2019
1,Bills,10,6,0,0.625,4,314.0,2019
2,Jets,7,9,0,0.438,-2,276.0,2019
3,Dolphins,5,11,0,0.313,-6,306.0,2019
4,Ravens,14,2,0,0.875,12,531.0,2019


In [92]:
#Combine the dataframe and create a wrangled/cleaned csv file

#I used pd.merge I merged each observation by Team and Year. 
nfl_WR_RB_df = pd.merge(season_df, top_salaries, on = ["Team", "Year"], how = "inner")

nfl_WR_RB_df.head(20)

,Team,W,L,T,win_pct,win_loss_diff,PF,Year,Player,Cap Number,Cash Spent,Position
0,Patriots,12,4,0,0.750,8,420.0,2019,James White,4625000,3218750,running-back
1,Patriots,12,4,0,0.750,8,420.0,2019,Julian Edelman,6077615,11000000,wide-receiver
2,Bills,10,6,0,0.625,4,314.0,2019,Frank Gore,2000000,2000000,running-back
3,Bills,10,6,0,0.625,4,314.0,2019,Cole Beasley,6900000,11368750,wide-receiver
4,Jets,7,9,0,0.438,-2,276.0,2019,Le'Veon Bell,8968750,14468750,running-back
5,Jets,7,9,0,0.438,-2,276.0,2019,Jamison Crowder,6781250,9000000,wide-receiver
6,Dolphins,5,11,0,0.313,-6,306.0,2019,Myles Gaskin,509895,575226,running-back
7,Dolphins,5,11,0,0.313,-6,306.0,2019,DeVante Parker,6900000,14500000,wide-receiver
8,Ravens,14,2,0,0.875,12,531.0,2019,Mark Ingram,3333333,6000000,running-back
9,Ravens,14,2,0,0.875,12,531.0,2019,Marquise Brown,2143153,7087612,wide-receiver


In [93]:
nfl_WR_RB_df.to_csv("nflWrRb_data.csv")